`terminal` - zmiena należąca do [Unicode Private Use Area](https://en.wikipedia.org/wiki/Private_Use_Areas), w ogólności powinno wystarczyć nawet na tekst z emoji.

In [1]:
import time
terminal = chr(0xF0000) # unique last symbol

# Trie

In [2]:
class Trie:
    class _Node:
        def __init__(self, letter, parent=None, children=None, link=None, pref_len=0):
            self.letter = letter
            self.parent = parent
            self.children = {} if not children else children
            self.link = link
            self.len = pref_len
        
        def add(self, child):
            if child.letter in self.children:
                raise RuntimeError('Undefined behavior')
            self.children[child.letter] = child
        
        def __getitem__(self, key):
            return self.children[key]
        
        def __contains__(self, key):
            return key in self.children
        
        def __len__(self):
            return self.len
        
    def __init__(self, text=None):
        self.root = self._Node(letter='', pref_len=0)
        self.root.link = self.root
        self.root.parent = None
        
        if text:
            self.build(text)
    
    def _validate_and_add(self, text):
        if text[-1] == terminal:
            return text
        else:
            return text + terminal
        
    def build(self, text):
        text = self._validate_and_add(text)
        head = self.root
        
        for i in range(len(text)):
            #if (i+1) % 1000 == 0: print(i+1, '/', len(text))
            leaf = self.add(text, i, head)
            head = self.up_link_down(leaf)
    
    def add(self, text, suffix_start, head):
        leaf = head
        for i in range(suffix_start+len(head), len(text)):
            new_node = self._Node(parent = leaf, letter=text[i], pref_len=len(leaf)+1)
            leaf.add(new_node)
            leaf = new_node
            
        return leaf
    
    def up_link_down(self, leaf):
        q = [] # works A LOT faster than LifoQueue (on 10^6 elements x30 faster)
        prev_branch = leaf
        while prev_branch.link is None:
            q.append(prev_branch.letter)
            prev_branch = prev_branch.parent
        
        new_head = prev_branch.link
        if prev_branch is self.root:
            l = q.pop()
            self.root[l].link = self.root
            prev_branch = self.root[l]
        
        while q and q[-1] in new_head:
            l = q.pop()
            prev_branch = prev_branch[l]
            new_head = new_head[l]
            prev_branch.link = new_head
        return new_head
    
    def find_prefix(self, text, suffix_start, start_node=None): # used erlier
        node = start_node if start_node else self.root
        for i in range(suffix_start, len(text)):
            if text[i] not in node:
                return node, i
            else: 
                node = node[text[i]]
        return node, len(text)
        
    def __contains__(self, suffix):
        return self.find_prefix(suffix, 0)[1] == len(suffix)

**krótki opis** - algorytm tworzy oddzielny `Node` dla każdej litery, `link`'i są tworzone w momencie poszukiwania następnej głowy (`head`) i służą do szybszego jej znalezienia, każdy link prowadzi od prefiksu $a\sigma$ do $\sigma$, gdie $a$ - dowolny pojedynczy znak, $\sigma$ - dowolny w tym pusty string.

# Degraded McCraight

In [3]:
class DegradedSuffixTree:
    class _Node:
        def __init__(self, letter=None, suffix_start=-1, pref_len=-1, parent=None, children=None):
            self.letter = '' if not letter else letter
            self.parent = parent
            self.start = suffix_start
            self.len = pref_len
            self.children = {} if not children else children
            self.suffix_link = None
        
        def __len__(self): # depth in tree, but I do it my way XD
            return self.len # length of string builded from root to the and of this node 
        
        def __contains__(self, key):
            return key in self.children
        
        def __getitem__(self, key):
            return self.children[key]
        
        def add(self, child):
            self.children[child.letter] = child
        
    
    def __init__(self, text=None):
        self.root = self._Node(suffix_start=0, pref_len=0)
        self.root.suffix_link = self.root
        self.root.parent = self.root
        
        if text is not None:
            self.build(text)
    
    def _validate_and_add(self, text):
        if text[-1] == terminal:
            return text
        else:
            return text + terminal
        
    def build(self, text):
        text = self._validate_and_add(text)
        
        head = self.root
        pref_len = 0 # length of matched prefix of i'th suffix 
        
        for i in range(len(text)):
            if pref_len == len(head) and text[i + pref_len] in head: ##TODO change
                head, pref_len = self.slow_find(text, i, head, pref_len)
            
            if len(head) > pref_len:
                head = self.split_node(text, head, pref_len)
            self.create_leaf(text, i, head, pref_len)
            
            pref_len = 0 # generaly max(0, pref_len - 1) 'cause of Lemma 1 in original paper
            head = self.root
            pref_len = 0
            
        self.text = text
    
    def slow_find(self, text, cur_start, head, pref_len):
        # jump to next node
        while pref_len == len(head) and text[cur_start + pref_len] in head:
            head = head[text[cur_start + pref_len]]
            pref_len += 1
            # go until the end of node searching for new head
            while pref_len < len(head) and text[cur_start + pref_len] == text[head.start + pref_len]:
                pref_len += 1
        return head, pref_len
    
    def split_node(self, text, node, pref_len):
        parent = node.parent
        new_node = self._Node(suffix_start=node.start, pref_len=pref_len, 
                              letter=text[node.start + len(parent)], parent=parent)
        parent.add(new_node)
        node.parent = new_node
        node.letter = text[node.start + pref_len]
        new_node.add(node)
        
        return new_node
    
    def create_leaf(self, text, suffix_start, head, pref_len):
        leaf = self._Node(suffix_start=suffix_start, pref_len=len(text)-suffix_start,
                          letter=text[suffix_start+pref_len])
        leaf.parent = head
        head.add(leaf)
        return leaf # just in case
    
    def __contains__(self, key):
        node = self.root
        
        for d, l in enumerate(key):
            if d == len(node):
                if l not in node:
                    return False
                node = node[l]
            else:
                if l != self.text[node.start + d]:
                    return False
        return True

**krótki opis** - algorytm McCraight'a bez używania elementów `link` oraz metody `fast_find`. Testy niżej pokazują że znacznie pogorsza to czas działania (algorytm nie jest liniowy względem danych wejściowych, bo nie możemy gwarantować stałej złożoności znalezienia następnej główy)

# McCraight algorithm

In [4]:
class SuffixTree:
    class _Node:
        def __init__(self, letter=None, suffix_start=-1, pref_len=-1, parent=None, children=None):
            self.letter = '' if not letter else letter
            self.parent = parent
            self.start = suffix_start
            self.len = pref_len
            self.children = {} if not children else children
            self.suffix_link = None
        
        def __len__(self): # depth in tree, but I do it my way XD
            return self.len # length of string builded from root to the and of this node 
        
        def __contains__(self, key):
            return key in self.children
        
        def __getitem__(self, key):
            return self.children[key]
        
        def add(self, child):
            self.children[child.letter] = child
        
    
    def __init__(self, text=None):
        self.root = self._Node(suffix_start=0, pref_len=0)
        self.root.suffix_link = self.root
        self.root.parent = self.root
        
        if text is not None:
            self.build(text)
    
    def _validate_and_add(self, text):
        if text[-1] == terminal:
            return text
        else:
            return text + terminal
        
    def build(self, text):
        text = self._validate_and_add(text)
        
        head = self.root
        pref_len = 0 # length of matched prefix of i'th suffix 
        
        for i in range(len(text)):
            if pref_len == len(head) and text[i + pref_len] in head:
                head, pref_len = self.slow_find(text, i, head, pref_len)
            
            if len(head) > pref_len:
                head = self.split_node(text, head, pref_len)
            self.create_leaf(text, i, head, pref_len)
            
            if head.suffix_link is None:
                self.fast_find(text, head, pref_len)
            head = head.suffix_link
            pref_len = len(head) # generaly max(0, pref_len - 1) 'cause of Lemma 1 in original paper
        
        self.text = text
    
    def slow_find(self, text, cur_start, head, pref_len):
        # jump to next node
        while pref_len == len(head) and text[cur_start + pref_len] in head:
            head = head[text[cur_start + pref_len]]
            pref_len += 1
            # go until the end of node searching for new head
            while pref_len < len(head) and text[cur_start + pref_len] == text[head.start + pref_len]:
                pref_len += 1
        return head, pref_len
    
    def split_node(self, text, node, pref_len):
        parent = node.parent
        new_node = self._Node(suffix_start=node.start, pref_len=pref_len, 
                              letter=text[node.start + len(parent)], parent=parent)
        parent.add(new_node)
        node.parent = new_node
        node.letter = text[node.start + pref_len]
        new_node.add(node)
        
        return new_node
    
    def create_leaf(self, text, suffix_start, head, pref_len):
        leaf = self._Node(suffix_start=suffix_start, pref_len=len(text)-suffix_start,
                          letter=text[suffix_start+pref_len])
        leaf.parent = head
        head.add(leaf)
        return leaf # just in case
    
    def fast_find(self, text, head, pref_len):
        next_head = head.parent.suffix_link
        
        while len(next_head) < pref_len - 1:
            next_head = next_head[text[head.start + len(next_head) + 1]]
        if len(next_head) > pref_len - 1:
            next_head = self.split_node(text, next_head, pref_len - 1)
        head.suffix_link = next_head
    
    def __contains__(self, key):
        node = self.root
        
        for d, l in enumerate(key):
            if d == len(node):
                if l not in node:
                    return False
                node = node[l]
            else:
                if l != self.text[node.start + d]:
                    return False
        return True

**krótki opis** - algotyrm konstrukcji drzewa wszystkich sufiksów przedstawiony [Edwardem McCraightem, w 1976](https://dl.acm.org/doi/10.1145/321941.321946). Zapewnia liniową złożoność budowania drzewa, i ~15% mniejszą złożoność pamięciową w porównaniu z innymi liniowymi metodami  

## Tests

In [13]:
import random
TEST1 = 'bbb$'
TEST2 = 'aabbabd'
TEST3 = 'ababcd'
TEST4 = 'abcbccd'
max_test_num = 10000
TEST5 = 'b'*5 + 'aba' + 'b'*3 + 'a'*2 + 'b'*5 + 'c'
with open('1997_714.txt') as f:
    TEST6 = f.read()

def test(test_class, num_tests=None):
    num_tests = num_tests if num_tests else 5
    for test_num, s in enumerate([TEST1, TEST2, TEST3, TEST4, TEST5, TEST6][:num_tests]):
        print(f'Building tree for TEST{test_num+1}')
        print('Build time:')
        %timeit index = test_class(s)
        index = test_class(s)
        n = min(max_test_num, len(s))
        if max_test_num < len(s):
            for i in range(0, n):
                r = random.randint(0, len(s))
                t = s[r:]
                if i % 1000 == 0:
                    print(f'search time for suffix len: {len(t)}')
                    %timeit t in index
                
                if not t in index:
                    return False
        else:
            for i in range(0, n):
                t = s[i:]
                if i % 1000 == 0:
                    print(f'search time for suffix len: {len(t)}')
                    %timeit t in index
                
                if not t in index:
                    return False
            
        if s is TEST5:
            if TEST1 in index:
                return False
        else:
            if TEST5 in index:
                return False
        print(f'Test num {test_num+1} passed')
    return True

In [14]:
if test(Trie, 5): # test(Trie, 6) U can do this but this takes eternity to build and a bit MORE then all your RAM  
    print('Tests passed') 
else:
    print('Tests failed')

Building tree for TEST1
Build time:
28.5 µs ± 363 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
search time for suffix len: 4
2.09 µs ± 64.4 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
Test num 1 passed
Building tree for TEST2
Build time:
63.9 µs ± 2.47 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
search time for suffix len: 7
3.17 µs ± 161 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
Test num 2 passed
Building tree for TEST3
Build time:
55.1 µs ± 6.18 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
search time for suffix len: 6
2.68 µs ± 7.58 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
Test num 3 passed
Building tree for TEST4
Build time:
68.1 µs ± 11.8 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
search time for suffix len: 7
3.13 µs ± 103 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
Test num 4 passed
Building tree for TEST5
Build time:
282 µs ± 15.8 µs per loop (mean ± s

In [15]:
if test(DegradedSuffixTree, num_tests=6):
    print('Tests passed')
else:
    print('Tests failed')

Building tree for TEST1
Build time:
18.4 µs ± 539 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
search time for suffix len: 4
2.15 µs ± 78.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
Test num 1 passed
Building tree for TEST2
Build time:
28.3 µs ± 235 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
search time for suffix len: 7
2.73 µs ± 34.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
Test num 2 passed
Building tree for TEST3
Build time:
21.4 µs ± 221 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
search time for suffix len: 6
2.43 µs ± 15.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
Test num 3 passed
Building tree for TEST4
Build time:
25.3 µs ± 884 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
search time for suffix len: 7
2.64 µs ± 109 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
Test num 4 passed
Building tree for TEST5
Build time:
102 µs ± 3.93 µs per loop (mean ± st

In [16]:
if test(SuffixTree, num_tests=6):
    print('Tests passed')
else:
    print('Tests failed')

Building tree for TEST1
Build time:
20.1 µs ± 195 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
search time for suffix len: 4
1.94 µs ± 10.9 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
Test num 1 passed
Building tree for TEST2
Build time:
31.2 µs ± 216 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
search time for suffix len: 7
2.71 µs ± 19.3 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
Test num 2 passed
Building tree for TEST3
Build time:
24 µs ± 213 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
search time for suffix len: 6
2.42 µs ± 16.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
Test num 3 passed
Building tree for TEST4
Build time:
27.3 µs ± 277 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
search time for suffix len: 7
2.56 µs ± 23.6 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
Test num 4 passed
Building tree for TEST5
Build time:
92.7 µs ± 280 ns per loop (mean ± std

# Wnioski

Jak widać z testów każde drzewo działa poprawnie, bo da się w nim znaleść każdy istniejący suffix. Dla `Trie` nie był przeprowadzony test na załączonym pliku (test numer 6), bo nie posiadam takiej ilości RAM. 1/20 tekstu po zbudowaniu na nim `Trie` zajmuje około 15 GB RAM. 

**Analiza czasu działania**:

* wszystkie struktury mają podobny czas wyszukiwania chociaż przez narzut przechodzenia do kolejnych dzieci `Trie` wydaje się być najwolniejszym.

* Czas budowania `Trie` rośnie drastycznie wraz z wzrostem rozmiaru tekstu. Teoretyczna złożoność czasawa algorytmu wynosi $O(N^{2})$, co sprawdza się w praktyce.

* Czas budowania `DegradedSuffixTree` (bez `fast_find` i `link`) w najgorszym przypadku wynosi $O(N^{2})$, ale testy pokazują że najgorszy przypadek w rzeczywistości nie zachodzi. Także struktura jest mniej pamięciożerna bo nie trzyma substringi, tylko indeksy początku i końca(długośc) w tekście zródłowym, czyli ne powiela kopij tekstu i nie trzyma każdy symbol w oddzielnym `Node`.

* Czas budowania `SuffixTree` metodą McCraight'a wynosi $O(N)$ co w rzeczywistości można zaobserwować patrząc na wynik ostatniego testu (2M symboli), czas wykonania jest ~4 razy mniejszy niż w przypadku bez używania `fast_find` i kilkaset+ razy szybszy od `Trie` (nie udalo się dostać wyników dla tak wielkiego tekstu).